# Lab-6 : GAN Practice

In [1]:
# Generative Adversarial Network(GAN) practice
# https://arxiv.org/abs/1406.2661
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
#########
# Setting the options
######
total_epoch = 100
batch_size = 100
learning_rate = 0.0002
# setting the number of nodes in the hidden layer and input layer
n_hidden = 256
n_input = 28 * 28
n_noise = 128  # setting the size of the noise as the input to the generator

In [3]:
#########
# Neural Network Constitution
######
# GAN is an unsupervised learning and has no labels.
X = tf.placeholder(tf.float32, [None, n_input])
# placeholder for the noise Z as the input
Z = tf.placeholder(tf.float32, [None, n_noise])

# Variables for the Generator.
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# Variables for the Discriminator.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))

D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [4]:
# Making the Generator Network(G).
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, G_W2) + G_b2)

    return output


# Making the Discriminator Network(D).
def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, D_W2) + D_b2)

    return output


# Making the random noise(Z).
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))


# Generate the image with the random noise.
G = generator(Z)
# Discriminate the generated image
D_gene = discriminator(G)
# Discriminate the real image
D_real = discriminator(X)

In [5]:
# loss function for the discriminator
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))

# loss function for the generator
loss_G = tf.reduce_mean(tf.log(D_gene))

# Variables for the loss_D
# Variables for the loss_G
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

# To maximize loss_D and loss_G
# minimize -loss_D and -loss_G
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D,
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G,
                                                         var_list=G_var_list)

In [ ]:
#########
# Training the Network
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        # Train the Discriminator and the Generator
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    #########
    # Save the images for every 10 epochs
    ######
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('End of optimization!')

Epoch: 0000 D loss: -0.5289 G loss: -2.013
Epoch: 0001 D loss: -0.4043 G loss: -2.191
Epoch: 0002 D loss: -0.6096 G loss: -1.654
Epoch: 0003 D loss: -0.3841 G loss: -1.598
